In [3]:
# Import libraries 
import requests
import pandas as pd
from datetime import datetime
import os
from dotenv import load_dotenv

In [4]:
# Define API details
load_dotenv()

X_API_KEY = os.getenv('X_API_KEY')
url = "https://orchestrator.pgatour.com/graphql"
headers = {
    "x-api-key": X_API_KEY
}

In [5]:
# Data to scrape
stats_to_fetch = {
    "120": "avg_score",
    "138": "top_10",
    "101": "drive_dist",
    "102": "drive_acc",
    "119": "putts_per_round",
    "130": "scramble",
    "103": "gir",
    "160": "bounce_back",
    "02675": "strokes_gained",
    "02567": "strokes_gained_ot",
    "02569": "strokes_gained_arg",
    "02674": "strokes_gained_ttg",
    "02568": "strokes_gained_atg",
    "02564": "strokes_gained_putt",
    "142": "par3_score",
    "143": "par4_score",
    "144": "par5_score"
}

In [6]:
# Function to scrape multiple stats and years
def fetch_player_stat(stat_id, year):
    payload = {
        "operationName": "StatDetails",
        "variables": {
            "tourCode": "R",
            "statId": stat_id,
            "year": year,
            "eventQuery": None
        },
        "query": """
            query StatDetails($tourCode: TourCode!, $statId: String!, $year: Int, $eventQuery: StatDetailEventQuery) {
            statDetails(
                tourCode: $tourCode
                statId: $statId
                year: $year
                eventQuery: $eventQuery
            ) {
                rows {
                ... on StatDetailsPlayer {
                    playerId
                    playerName
                    country
                    rank
                    stats {
                        statName
                        statValue
                    }
                }
                }
            }
            }
        """
    }   

    response = requests.post(url, json=payload, headers=headers)
    if response.status_code != 200:
        print(f"API error for {stat_id} ({year}): {response.status_code}")
        return []

    data = response.json()
    rows = data.get('data', {}).get('statDetails', {}).get('rows', [])
    
    player_stats = []
    for row in rows:
        if 'playerName' in row:
            # Keep raw values as-is
            for s in row['stats']:
                player_stats.append({
                    'player_id': row['playerId'],
                    'player_name': row['playerName'],
                    'country': row.get('country'),
                    'year': year,
                    'rank': row.get('rank'),
                    'stat_id': stat_id,
                    'stat_name': stats_to_fetch.get(stat_id, f"stat_{stat_id}"),
                    'stat_value': s['statValue'],  # Keep raw string
                    'scraped_at': datetime.now()
                })
    
    print(f"Fetched {len(player_stats)} rows for {stat_id} ({year})")
    return player_stats

In [7]:
# Define years to scrape
start_year = datetime.now().year - 20
end_year = datetime.now().year

# To store all data
all_stats = []

# Nested Loop through years and stat's
for year in range(start_year, end_year + 1):
    for stat_id in stats_to_fetch.keys():
        all_stats.extend(fetch_player_stat(stat_id, year))

print(f'All data scraped between {start_year} and {end_year}')

df = pd.DataFrame(all_stats)

Fetched 808 rows for 120 (2005)
Fetched 748 rows for 138 (2005)
Fetched 606 rows for 101 (2005)
Fetched 606 rows for 102 (2005)
Fetched 808 rows for 119 (2005)
Fetched 606 rows for 130 (2005)
Fetched 808 rows for 103 (2005)
Fetched 202 rows for 160 (2005)
Fetched 1010 rows for 02675 (2005)
Fetched 606 rows for 02567 (2005)
Fetched 606 rows for 02569 (2005)
Fetched 1010 rows for 02674 (2005)
Fetched 606 rows for 02568 (2005)
Fetched 606 rows for 02564 (2005)
Fetched 606 rows for 142 (2005)
Fetched 606 rows for 143 (2005)
Fetched 606 rows for 144 (2005)
Fetched 784 rows for 120 (2006)
Fetched 740 rows for 138 (2006)
Fetched 588 rows for 101 (2006)
Fetched 588 rows for 102 (2006)
Fetched 784 rows for 119 (2006)
Fetched 588 rows for 130 (2006)
Fetched 784 rows for 103 (2006)
Fetched 196 rows for 160 (2006)
Fetched 980 rows for 02675 (2006)
Fetched 588 rows for 02567 (2006)
Fetched 588 rows for 02569 (2006)
Fetched 980 rows for 02674 (2006)
Fetched 588 rows for 02568 (2006)
Fetched 588 rows

In [8]:
df.to_csv('golf_data.csv', index = False)